# Processing NFT Trait Value Occurrences into JSON Files

This notebook outlines the process for creating JSON files that store the occurrences of NFT trait values. These files are later used in [`calculations.ipynb`](./calculations.ipynb) to calculate trait value scores, rarity scores, and ranks.

If the JSON files for trait occurrences are missing, you can run this notebook to generate them. However, ensure you have the NFT data files (`tbc_roundone.json` and `tbc_roundtwo.json`) before proceeding. If these files are also missing, first run [`download_nft_data.ipynb`](download_nft_data.ipynb) to scrape and save the required data.


---

### 1.1 Open Statements

In [8]:
open System
open System.Collections.Generic
open System.IO
open System.Text.Json
open System.Reflection

### 1.2 Creating the Type for JSON Serialization
The `NftTraitData` type represents the traits of each NFT. It includes fields such as `Background`, `Bear`, `Clothes`, and others. This type is used to deserialize JSON data and dynamically access NFT properties during processing.


In [9]:
type NftTraitData = {
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

### 1.3 Calculate the Trait Value Occurrences
The function `countAttributes` reads the NFT JSON file, deserializes its contents, and calculates the number of occurrences for each trait value across all NFTs. It dynamically accesses the properties of the `NftTraitData` type, returning a nested dictionary of the form Dictionary<string, Dictionary<string, int>>:

- Outer dictionary: Maps trait categories (e.g., `Background`, `Bear`) to inner dictionary
- Inner dictionary: Maps each trait value to its occurrence count.

The output is used to understand the distribution of trait values within the dataset.

In [10]:
let countAttributes (jsonFilePath: string) =

    let jsonString = File.ReadAllText(jsonFilePath)
    let nftList = JsonSerializer.Deserialize<NftTraitData list>(jsonString)
    let attributeCounts = Dictionary<string, Dictionary<string, int>>()

    let nftAttributes = 
        typeof<NftTraitData>.GetProperties()
        |> Array.filter (fun prop -> prop.PropertyType = typeof<string>)

    for attribute in nftAttributes do
        let propertyName = attribute.Name
        for nft in nftList do
            let value = attribute.GetValue(nft) :?> string
            if not (isNull value) then
                if not (attributeCounts.ContainsKey(propertyName)) then
                    attributeCounts.[propertyName] <- Dictionary<string, int>()

                let propertyDict = attributeCounts.[propertyName]
                if not (propertyDict.ContainsKey(value)) then
                    propertyDict.[value] <- 0

                propertyDict.[value] <- propertyDict.[value] + 1

    attributeCounts

### 1.4 Serialize and Save Results
The function `serializeAttributes` takes in the output of `countAttributes`, serializes the data, and then saves it in the indicated file.

In [11]:
let serializeAttributes (attributeCounts: Dictionary<string, Dictionary<string, int>>) (outputFilePath: string) =
    try
        let directory = Path.GetDirectoryName(outputFilePath)
        if not (Directory.Exists(directory)) then
            Directory.CreateDirectory(directory) |> ignore

        let jsonOutput = JsonSerializer.Serialize(attributeCounts, JsonSerializerOptions(WriteIndented = true))
        File.WriteAllText(outputFilePath, jsonOutput)
        printfn "Attribute counts saved to %s" outputFilePath
    with
    | ex ->
        printfn "Error during serialization: %s" ex.Message
        raise ex


### 1.5 Call the Functions for Each of the NFT Rounds

In [12]:
let jsonFilePath = "./data/tbc_roundone.json"
let outputPath = "./data/tbc_traits_roundone.json"
let attributeCounts = countAttributes jsonFilePath
serializeAttributes attributeCounts  outputPath

Attribute counts saved to ./data/tbc_traits_roundone.json


In [13]:
let jsonFilePath = "./data/tbc_roundtwo.json"
let outputPath = "./data/tbc_traits_roundtwo.json"
let attributeCounts = countAttributes jsonFilePath
serializeAttributes attributeCounts outputPath

Attribute counts saved to ./data/tbc_traits_roundtwo.json
